In [ ]:
# # MOVE TO ANALYSIS
# # Plot mean distribution of lattice occupation for different concentrations
# if simPa.Scan == 'Length':
    
#     fig, ax = plt.subplots(figsize = (5,5))

#     # Prepare data mean density
#     for num in range(simPa.nScan):

#         #  Prepare data filter
#         if simPa.Scan is not False:
#             filter_scan = SIM_results[simPa.Scan]==var_param[num]
#             lattice = SIM_results[filter_scan].Lattice        
#         else:
#             lattice = SIM_results.Lattice 

#          # Prepare data
#         X = np.arange(0, lattice.iloc[0].shape[1])
#         X = X * (simPa.step/1000) # change units to um
#         Y = np.mean(lattice, axis=0)     

#         ax.plot(X,Y[-1,])

#     ax.set_xlabel('Microtubule length ($\mu$m)', fontsize=13)
#     ax.set_ylabel('Motor density (a.u.)' , fontsize=13)
#     ax.set_title('State @ t=' + str(simPa.T_max) +' sec', fontsize=14)
    
#     if simPa.Scan is 'Length':
#         labels = ['%.0f $\mu$m' %(var_param[x]*(simPa.step/1000)) for x in range(simPa.nScan)]
#         ax.legend(labels, title=simPa.Scan, fontsize='small', loc='upper left')
# #     elif simPa.Scan is 'Concentration':
# #         labels = ['%.2f nM' %var_param[x] for x in range(simPa.nScan)]
# #         ax.legend(labels, title=simPa.Scan, fontsize='small', loc='upper left')     

#     # Save plot
#     if simPa.record_data:
#         filename = foldername + "/" + 'Figure_2'
#         plt.savefig(filename+'.eps', format='eps', dpi=1200)
#         plt.savefig(filename+'.png', format='png', dpi=300)

#     plt.show()

In [ ]:
# Plot of traffic jam formation

# fig, ax = plt.subplots(figsize = (5,5))

# for num in range(1):
# # for num in range(simPa.nScan):

#     # Prepare data filter            
#     if simPa.Scan is not False:
#         filter_scan = RESULTS[simPa.Scan]==var_param[num]
#         lattice = RESULTS[filter_scan].Lattice
#     else:                      
#         lattice = RESULTS.Lattice  

#     # Prepare data
#     time = np.arange(0, lattice.iloc[0].shape[0])
#     time = time * simPa.sampling # change units to seconds
#     intp = 10
#     Y = np.mean(lattice, axis=0)    
#     L = Y.shape[1]
    
#     for row in range(Y.shape[0]):
#         Y_intp = np.interp(time_intp, time, Y[row,])
#         FWHM[row] = np.where(Y[x,] >= 0.5)[0]
        
#     print(FWHM)
# print(L)
#     FWHM = [np.where(Y[x,] >= 0.5)[0] for x in range(Y.shape[0])]
#     for i in range(len(FWHM)):
#         if FWHM[i].size > 0:
#             FWHM[i] = abs(FWHM[i][0]-Y.shape[1])
#         else:
#             FWHM[i] = 0

    

#     ax.plot(time, FWHM)    

# ax.set_xlabel('Microtubule length ($\mu$m)', fontsize=12)
# ax.set_ylabel('FWHM (nm)' , fontsize=12)
# ax.set_title('Motor density', fontsize = 14)


# # Save plot
# if simPa.record_data:
#     filename = foldername + "/" + 'Figure_3'
#     plt.savefig(filename+'.eps', format='eps', dpi=1200)
#     plt.savefig(filename+'.png', format='png', dpi=300)

# plt.show()